In [1]:
from torch.nn.utils.rnn import pad_sequence
from espnet2.tasks.ssl import SSLTask
import soundfile as sf
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

xeus_model, xeus_train_args = SSLTask.build_model_from_file(
    None,
    'XEUS/model/xeus_checkpoint.pth',
    device,
)

c:\work\audio_to_text\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [9]:
!ffmpeg

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import moviepy
import moviepy.editor as ed
path = r'c:\Users\aakorobov\Videos\2024-07-15_11-07-23.mkv'

ac = ed.AudioFileClip(path)
ac.write_audiofile("out_audio_1.wav", ffmpeg_params=["-ac", "1"])

MoviePy - Writing audio in out_audio_1.wav


MoviePy - Done.


In [37]:
#wavs, sampling_rate = sf.read('out_audio.wav')
wavs.shape

torch.Size([1001070, 2])

In [2]:
from torch.utils.checkpoint import checkpoint
wavs, sampling_rate = sf.read('out_audio.wav') # sampling rate should be 16000
wav_lengths = torch.LongTensor([len(wav) for wav in [wavs]]).to(device)
wavs = pad_sequence(torch.Tensor(wavs)).to(device) 

# we recommend use_mask=True during fine-tuning
feats = xeus_model.encode(wavs, wav_lengths, use_mask=False, use_final_output=False)[0][-1] # take the output of the last layer -> batch_size x seq_len x hdim

KeyboardInterrupt: 

## TEST whisper 

In [ ]:
!ffmpeg

ffmpeg version 7.0.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame -

In [11]:
#whisper.load_audio?
import os
#os.path.isfile('out_audio.wav')
audio = whisper.load_audio("out_audio.wav")

In [15]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("out_audio_1.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio)#.to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)
#Команды оптимизаторы, Николая и Дин. Лучше, конечно, ничего. Подождать. Сейчас еще раз уберу.


Detected language: ru
Команды оптимизаторы, Николая и Дин. Лучше, конечно, ничего. Подождать. Сейчас еще раз уберу.


In [14]:
options = whisper.DecodingOptions()
vars(options)

{'task': 'transcribe',
 'language': None,
 'temperature': 0.0,
 'sample_len': None,
 'best_of': None,
 'beam_size': None,
 'patience': None,
 'length_penalty': None,
 'prompt': None,
 'prefix': None,
 'suppress_tokens': '-1',
 'suppress_blank': True,
 'without_timestamps': False,
 'max_initial_timestamp': 1.0,
 'fp16': True}

In [13]:
import whisper

model = whisper.load_model("medium")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("out_audio_1.wav")
audio = whisper.pad_or_trim(audio)


# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio)#.to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)
#Команды оптимизаторы, Николая и Дин. Лучше, конечно, ничего. Подождать. Сейчас еще раз уберу.
#команды оптимизатора Николая Дин. Лучше, конечно, начало подождать. Сейчас еще раз наберу.


100%|█████████████████████████████████████| 1.42G/1.42G [10:30<00:00, 2.42MiB/s]


Detected language: ru
команды оптимизатора Николая Дин. Лучше, конечно, начало подождать. Сейчас еще раз наберу.


In [28]:
audio = whisper.load_audio(r"c:\Users\aakorobov\Videos\2024-05-29 14-18-32.mkv")
#audio = whisper.load_audio(r"c:\Users\aakorobov\Videos\audio\2024-05-29 14-18-32.wav")
audio1 = whisper.pad_or_trim(audio)

In [36]:
model.transcribe?

Signature:
model.transcribe(
    audio: Union[str, numpy.ndarray, torch.Tensor],
    *,
    verbose: Optional[bool] = None,
    temperature: Union[float, Tuple[float, ...]] = (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    compression_ratio_threshold: Optional[float] = 2.4,
    logprob_threshold: Optional[float] = -1.0,
    no_speech_threshold: Optional[float] = 0.6,
    condition_on_previous_text: bool = True,
    initial_prompt: Optional[str] = None,
    word_timestamps: bool = False,
    prepend_punctuations: str = '"\'“¿([{-',
    append_punctuations: str = '"\'.。,，!！?？:：”)]}、',
    **decode_options,
)
Docstring:
Transcribe an audio file using Whisper

Parameters
----------
model: Whisper
    The Whisper model instance

audio: Union[str, np.ndarray, torch.Tensor]
    The path to the audio file to open, or the audio waveform

verbose: bool
    Whether to display the text being decoded to the console. If True, displays all the details,
    If False, displays minimal details. If None, does not di

In [30]:
print(type(audio),type(audio1))
print(audio.shape,audio1.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2684587,) (480000,)


In [32]:
import whisper
model = whisper.load_model("base")

In [35]:
result

{'text': ' Вот просто вот так этому файбу. Вот в Парсинге, скажите, Алексей, вот к облицам, которым мы заберем данные, они в таком же формате, как файбе будет лежать, и нам какие-то преобразования делать. То есть условно, к облицам будет два поля, тайп и что там еще не вэлью. Все больше там лучше не будет. Или как? Ну, все правильно, больше ничего не должно быть. Значение, которое там будет, это будет 6он. Мы просто скрипты. Заносим, читаем его оттуда. Этот 6он сам дикарализуется в объект. И все значения в тортам есть уже используется с криптием. Окей, то есть, грубо говоря, на нашей стороне это будет политакт. Что мы делаем у нас запрос к Парсингу, к облицу Парсинга. На выходе получаем, сколько будет одна строчка, я так понимаю, или сколько. Да, одна, одна. Быта одна идиная строчка. Игения, я отписал до этого, что мы планируем этот облиц и переспользоваться для дальнейших парсисов. Ну, ну, давайте это сейчас пока. Судем то, что с ресторанами. Да, поэтому одна строчка при условии, если

In [34]:
result = model.transcribe(r"c:\Users\aakorobov\Videos\2024-05-29 14-18-32.mkv")
result['text']

' Вот просто вот так этому файбу. Вот в Парсинге, скажите, Алексей, вот к облицам, которым мы заберем данные, они в таком же формате, как файбе будет лежать, и нам какие-то преобразования делать. То есть условно, к облицам будет два поля, тайп и что там еще не вэлью. Все больше там лучше не будет. Или как? Ну, все правильно, больше ничего не должно быть. Значение, которое там будет, это будет 6он. Мы просто скрипты. Заносим, читаем его оттуда. Этот 6он сам дикарализуется в объект. И все значения в тортам есть уже используется с криптием. Окей, то есть, грубо говоря, на нашей стороне это будет политакт. Что мы делаем у нас запрос к Парсингу, к облицу Парсинга. На выходе получаем, сколько будет одна строчка, я так понимаю, или сколько. Да, одна, одна. Быта одна идиная строчка. Игения, я отписал до этого, что мы планируем этот облиц и переспользоваться для дальнейших парсисов. Ну, ну, давайте это сейчас пока. Судем то, что с ресторанами. Да, поэтому одна строчка при условии, если будет сд